<a href="https://colab.research.google.com/github/ufozone/aiounifi/blob/master/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [1]:
# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.12.12
PyTorch: 2.8.0+cu126
CUDA available: True
Sat Nov 15 11:13:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [2]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,143 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,181 kB]
Get:12 https://cli.github.com/packages stable/main amd64 Packages [343 B]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/pp

2) Clone repo fresh

In [4]:
%cd /content
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!pwd

/content
Cloning into 'piper1-gpl'...
remote: Enumerating objects: 817, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 817 (delta 176), reused 157 (delta 155), pack-reused 598 (from 4)
Receiving objects: 100% (817/817), 4.69 MiB | 30.81 MiB/s, done.
Resolving deltas: 100% (455/455), done.
/content/piper1-gpl
/content/piper1-gpl


3) Python deps (editable install, no venv in Colab)

In [5]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
Obtaining file:///content/piper1-gpl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/82

4) Build the Cython extension used for alignment

In [6]:
%cd /content/piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

/content/piper1-gpl
Compiling /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
P

5) Dev build (repo mode)

In [7]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

  Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
  Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.5 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached scikit_build-0.18.1-py3-none-any.whl (85 kB)
Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (29.7 MB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Attempting uninstall: cmake
    Found existing installation: cmake 3.31.6
    Uninstalling cmake-3.31.6:
      Successfully uninstalled cmake-3.31.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [cmake]


In [8]:
%cd /content/piper1-gpl
!python3 setup.py build_ext --inplace -v

/content/piper1-gpl


--------------------------------------------------------------------------------
-- Trying 'Ninja' generator
--------------------------------
---------------------------
----------------------
-----------------
------------
-------
--
Not searching for unused variables given on the command line.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX

6) (Optional) Mount Google Drive for datasets and outputs

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


7) Set paths and training hyperparams

In [25]:
from pathlib import Path

# ==== CHANGE THESE ====
VOICE_NAME      = "my_colab_voice"
ESPEAK_VOICE    = "de"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050
BATCH_SIZE      = 32       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path("/content/drive/MyDrive/tts_data/myvoice")
AUDIO_DIR       = DATA_ROOT / "wavs"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path("/content/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

CSV exists: True
Audio dir exists: True
Cache dir: /content/piper_cache
Config will be written to: /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json


8) Quick sanity checks

In [26]:
!espeak-ng --voices | head -n 20

Pty Language       Age/Gender VoiceName          File                 Other Languages
 5  af              --/M      Afrikaans          gmw/af               
 5  am              --/M      Amharic            sem/am               
 5  an              --/M      Aragonese          roa/an               
 5  ar              --/M      Arabic             sem/ar               
 5  as              --/M      Assamese           inc/as               
 5  az              --/M      Azerbaijani        trk/az               
 5  ba              --/M      Bashkir            trk/ba               
 5  bg              --/M      Bulgarian          zls/bg               
 5  bn              --/M      Bengali            inc/bn               
 5  bpy             --/M      Bishnupriya_Manipuri inc/bpy              
 5  bs              --/M      Bosnian            zls/bs               
 5  ca              --/M      Catalan            roa/ca               
 5  cmn             --/M      Chinese_(Mandarin) sit/cmn    

In [27]:
import pandas as pd, io, os, textwrap

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

                         audio  \
0  Frank_Schaff_Hoerspiel_1_01   
1  Frank_Schaff_Hoerspiel_1_02   
2  Frank_Schaff_Hoerspiel_1_03   
3  Frank_Schaff_Hoerspiel_1_04   
4  Frank_Schaff_Hoerspiel_1_05   

                                                text  
0                       hallo, ich bin frank schaff.  
1  und ich öffne mit euch das nächste türchen im ...  
2               ihr hört einen text von manuel dörr.  
3  wie schwer kann es schon sein, eine fernsehser...  
4   sieh dir doch den ganzen schund im fernsehen an.  
Missing among first 5: ['Frank_Schaff_Hoerspiel_1_01', 'Frank_Schaff_Hoerspiel_1_02', 'Frank_Schaff_Hoerspiel_1_03', 'Frank_Schaff_Hoerspiel_1_04', 'Frank_Schaff_Hoerspiel_1_05']


9) Kick off training

In [ ]:
!python3 -m piper.train fit \
  --data.voice_name "my_colab_voice" \
  --data.csv_path "/content/drive/MyDrive/tts_data/myvoice/metadata.csv" \
  --data.audio_dir "/content/drive/MyDrive/tts_data/myvoice/wavs" \
  --model.sample_rate 22050 \
  --data.espeak_voice "de" \
  --data.cache_dir "/content/piper_cache" \
  --data.config_path "/content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json" \
  --data.batch_size 8 \
  --ckpt_path "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/de/de_DE/thorsten/medium/epoch%3D3135-step%3D2702056.ckpt"

/usr/local/lib/python3.12/dist-packages/lightning/fabric/utilities/seed.py:44: No seed found, seed set to 0
Seed set to 0
/usr/local/lib/python3.12/dist-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
INFO:piper.train.vits.dataset:Processed 154 utterance(s)
2025-11-15 11:30:28.644356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763206228.661665    7367 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attemp

10. Export to onnx

In [ ]:
!python3 -m piper.train.export_onnx \
  --checkpoint "/content/piper1-gpl/lightning_logs/version_2/checkpoints/epoch=2174-step=2680.ckpt" \
  --output-file "/content/model.onnx"

In [ ]:
!cp /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json /content/model.onnx.json